In [1]:
from __future__ import print_function, division
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")


from sklearn.preprocessing import MultiLabelBinarizer

pd.set_option('display.max_columns', 30)

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

# Load data

In [8]:
def memory(df):
    print("Memory usage of the dataframe is {:.2f} MB".format(
        df.memory_usage().sum() / 1024**2))

In [10]:
convert = False

if(convert):
    df = pd.read_csv('../we_data/train.csv')
    df.to_hdf('train.h5', 'train')
    
else:
    df = pd.read_hdf('train.h5', 'train')

In [11]:
df['actualpayprice'] = df['payprice']/1000
df = df.fillna(0)

In [12]:
memory(df)

Memory usage of the dataframe before 500.77 MB
Memory usage of the dataframe after 340.80 MB, redution of 31.94%


In [13]:
df.head()

,click,weekday,hour,bidid,userid,useragent,IP,region,city,adexchange,domain,url,urlid,slotid,slotwidth,slotheight,slotvisibility,slotformat,slotprice,creative,bidprice,payprice,keypage,advertiser,usertag,actualpayprice
0,0,5,22,b7bea80521fdecd95d2d761a38c91c3f09618066,2e880fb7d690cf7377b2e42e701728e3f3c0e4c1,windows_ie,125.37.175.*,2,2,2.0,trqRTvKaXTKfgg24JKTI,6447a7dfa30fe6eb410c91860e7c9b45,0.0,2015392487,200,200,2,0,5,a4f763f78ef3eedfe614263b94a8924e,238,5,0f951a030abdaedd733ee8d114ce2944,3427,0,0.005
1,0,1,20,4f51205475678f5a124bc76b2c54163bf8eaa7eb,3a1fe01360ff8100e7d006b83b77a3e4c01d928c,windows_chrome,171.36.92.*,238,239,1.0,20fc675468712705dbf5d3eda94126da,3ddf173a94bd23c326683b6373c75dd4,0.0,mm_10982364_973726_8930541,300,250,FourthView,Na,0,10722,294,23,0,2821,0,0.023
2,0,3,13,b604e3fd054a658ab7ced4285ebf2ef54d2bd890,801d18a056b6fe6b06a794aef17fb0d6daff2414,windows_ie,59.46.106.*,40,41,2.0,trqRTJn7O95I1mKYUV,625d1b5916ea925332c7b326c0574cfa,0.0,1720123646,250,250,2,0,5,798b2d49952d77f1eace9f23c210d0b5,238,24,0f951a030abdaedd733ee8d114ce2944,3427,"10052,10006,13866,10110",0.024
3,0,6,23,0348beeae93e561584c3b50fc9e7746a33048ad7,0d6eaf2259699990e38a1fc5116f112070b9ecdc,windows_ie,114.250.226.*,1,1,1.0,5F97t5E0BTK7XhNrUMpENpn,dedc488b98ca20707bc9a723957e7d1f,0.0,mm_10027070_118039_10308280,160,600,2,1,0,cb7c76e7784031272e37af8e7e9b062c,300,25,bebefa5efe83beee17a3d245e7c5085b,1458,"13866,10063,10111",0.025
4,0,5,6,268149c1789bce2bc9798ffd97ec431219bafeb3,a239d9bb642460d974ba67f85e63b8d3e214da0e,windows_ie,183.63.192.*,216,233,2.0,13625cb070ffb306b425cd803c4b7ab4,4199d1227c511fc776b76594dabff9f8,0.0,1120200150,728,90,OtherView,Na,133,7330,277,133,0,2259,0,0.133


In [14]:
df.shape

(2430981, 26)

# Problem 1

In [17]:
no_click,click = df['click'].value_counts().values
print('Baseline CTR {:.5f}'.format(click/no_click))

Baseline CTR 0.00074


In [18]:
def calcluate_num_impressions(df, grouping = False):
    imp = {}
    
    if(grouping):
        imp = df.groupby('weekday').agg({'click': 'count'}).to_dict()['click']
            
    else:
        imp = len(df)
    
    print(imp)
    return imp
    


def calcluate_num_clicks(df, grouping = False):
    clicks = {}
    
    if(grouping):
        for index, group in df.groupby(grouping):
            clicks[index] = np.sum(group['click'] ==1)
            
    else:
        clicks = np.sum(df['click'] == 1)
    
    print(clicks)
    return clicks
    

def calcluate_ctr(df, grouping = False):
    ctr = {}
    
    if(grouping):
        for index, group in df.groupby(grouping):
            ctr[index] = np.sum(group['click'] ==1)/ np.sum(group['click'] ==0)
            
    else:
        ctr = np.sum(df['click'] ==1)/ np.sum(df['click'] ==0)
    
    print(ctr)
    return ctr
    
    
def average_cost_per_mille(df, grouping = False):
    cpm = {}
    
    if(grouping):
        cpm = df.groupby('weekday').agg({'payprice': np.mean}).to_dict()['payprice']
        
    else:
        cpm = np.mean(df['payprice'])
        
    print(cpm)
    return cpm

In [19]:
average_cost_per_mille(df)

78.15141623895867

In [20]:
calcluate_num_clicks(df, 'weekday')

{0: 271, 1: 249, 2: 286, 3: 283, 4: 264, 5: 247, 6: 193}

In [21]:
calcluate_num_impressions(df, 'weekday')

{0: 420955, 1: 368164, 2: 296438, 3: 339627, 4: 326018, 5: 340642, 6: 339137}

# Problem 2

In [ ]:
pd.read_csv('we_data/validation.csv')

# Problem 3

In [24]:
df_copy = df.copy(deep=True)

df_copy = df_copy.drop(columns=['bidprice', 'payprice', 'actualpayprice'])
df_copy = df_copy.drop(columns=['urlid']) # seem to contain no info

# converting all fields to str is a quick hack for one hot to work properly
for column in df_copy.columns:
    df_copy[column] = df_copy[column].astype(str)

In [25]:
mlb = MultiLabelBinarizer()
# mlb.fit_transform(df_copy['usertag'].str.split(','))

df_copy = df_copy.join(
    pd.DataFrame(
        mlb.fit_transform(df_copy['usertag'].str.split(',')),
        columns='usertag_' + mlb.classes_,
        index=df_copy.index))
df_copy = df_copy.drop(columns='usertag')

In [26]:
mlb.classes_

array(['0', '10006', '10024', '10031', '10048', '10052', '10057', '10059',
       '10063', '10067', '10074', '10075', '10076', '10077', '10079',
       '10083', '10093', '10102', '10110', '10111', '10114', '10115',
       '10116', '10117', '10118', '10120', '10123', '10125', '10126',
       '10127', '10129', '10130', '10131', '10133', '10138', '10140',
       '10142', '10145', '10146', '10147', '10148', '10149', '10684',
       '11092', '11278', '11379', '11423', '11512', '11576', '11632',
       '11680', '11724', '11944', '13042', '13403', '13496', '13678',
       '13776', '13800', '13866', '13874', '14273', '15398', '16593',
       '16617', '16661', '16706', '16751', '16753'], dtype=object)

In [27]:
pd.get_dummies(df_copy.loc[:200])

,usertag_0,usertag_10006,usertag_10024,usertag_10031,usertag_10048,usertag_10052,usertag_10057,usertag_10059,usertag_10063,usertag_10067,usertag_10074,usertag_10075,usertag_10076,usertag_10077,usertag_10079,...,keypage_b2e35064f3549d447edbbdfb1f707c8c,keypage_bebefa5efe83beee17a3d245e7c5085b,keypage_c8758e8658b05bc3a88af7b37bdf0575,keypage_d29e59bf0f7f8243858b8183f14d4412,keypage_d41d8cd98f00b204e9800998ecf8427e,keypage_f4a09c06b67d0fcce088ee6e86ca052a,advertiser_1458,advertiser_2259,advertiser_2261,advertiser_2821,advertiser_2997,advertiser_3358,advertiser_3386,advertiser_3427,advertiser_3476
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
5,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,...,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
6,0,1,1,0,0,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
7,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
8,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
9,0,1,0,1,0,0,0,0,1,0,0,1,0,1,0,...,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
